In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/1.part_number'):
    shutil.rmtree('./file/1.part_number')

os.makedirs('./file/1.part_number')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./file/input.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:
            url = input_.loc[a, 'Url']

            dict_param = {}
            list_param = url.split('?')[1].split('&') if '?' in url else []
            for param in list_param:
                dict_param[param.split('=')[0]] = param.split('=')[1]
                
            output, page = pd.DataFrame(), 0
            while True:
                b = 0
                while True:
                    b += 1
                    
                    try:
                        request_url = f'''https://ecatalog.smpcorp.com/v2/fs/api/part/partsearch?filter={dict_param['part']}&filterType={dict_param.get('search', 'n')}&searchType={dict_param.get('type', 'null')}&imageSize=80&start={page*96}&limit=96&sort=3&catFilter=-All-&yearFilter=-All-&makeFilter=-All-&modelFilter=-All-&engineFilter=-All-&attrCodeFilter=-All-&attrValueFilter=-All-&plkEngineMakeFilter=-All-&plkEngineModelFilter=-All-&plkEngineDispFilter=-All-'''
                        
                        resp = requests.get(request_url,
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))

                        if resp.status_code == 200:
                            list_dict = resp.json()
                            if list_dict:
                                break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                if page == 0:
                    count = list_dict[0]['maxRows']

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame({'No': [dict_['rowId'] for dict_ in list_dict],
                                        'Part Number': [dict_['basePart'].strip() for dict_ in list_dict],
                                        'Url': [f'''https://www.4s.com/en/ecatalog?part={dict_['basePart']}&type=p&search=s''' for dict_ in list_dict]})

                output = pd.concat([output, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
    
                if work.qsize() % 100 == 0 and work.qsize() != 0:
                    clear_output()
    
                print(f'''[尝试次数：{b}] - {input_.loc[a, 'No']}.{request_url} > Page {page + 1}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')
    
                # = = = = = = = = = = = = = = =

                page += 1

                # = = = = = = = = = = = = = = =

                if len(output) == count:
                    output = output.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
                    
                    output.to_excel(f'''./file/1.part_number/crawler_{input_.loc[a, 'No']}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
                    
                    break
                    
            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
            df_temp.loc[0, 'Request_Url'] = request_url
            
            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel(f'''./file/part_number_error.xlsx''', index=False)
    print('爬虫存在error')
    print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/1.part_number'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/1.part_number/{file}',
              f'''./file/1.part_number/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：2

[尝试次数：1] - 1.https://ecatalog.smpcorp.com/v2/fs/api/part/partsearch?filter=Evaporator%20Core&filterType=s&searchType=p&imageSize=80&start=0&limit=96&sort=3&catFilter=-All-&yearFilter=-All-&makeFilter=-All-&modelFilter=-All-&engineFilter=-All-&attrCodeFilter=-All-&attrValueFilter=-All-&plkEngineMakeFilter=-All-&plkEngineModelFilter=-All-&plkEngineDispFilter=-All- > Page 1
[剩余数量：0] - [当前时间：11:22:00]

[尝试次数：1] - 2.https://ecatalog.smpcorp.com/v2/fs/api/part/partsearch?filter=Heater%20Core&filterType=s&searchType=p&imageSize=80&start=0&limit=96&sort=3&catFilter=-All-&yearFilter=-All-&makeFilter=-All-&modelFilter=-All-&engineFilter=-All-&attrCodeFilter=-All-&attrValueFilter=-All-&plkEngineMakeFilter=-All-&plkEngineModelFilter=-All-&plkEngineDispFilter=-All- > Page 1
[剩余数量：0] - [当前时间：11:22:00]

[尝试次数：1] - 2.https://ecatalog.smpcorp.com/v2/fs/api/part/partsearch?filter=Heater%20Core&filterType=s&searchType=p&imageSize=80&start=96&limit=96&sort=3&catFilter=-All-&yearFilter=-All-&makeFil

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 5011.12it/s]

Done ~
